## Part 4: Fraud

In [3]:
%run "../scripts/outlier.py" "../scripts/paths.json"

22/10/06 22:38:36 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


22/10/06 22:39:34 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [11]:
# %run "../scripts/fraud_feature.py" "../scripts/paths.json"

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=MAST30034 Project 2, master=local[*]) created by getOrCreate at /Users/ahirve/Desktop/ads_bnpl/scripts/ETL.py:21 

In [3]:
#Loading consumer fraud data: 
fraud_consumer = pd.read_csv('../data/tables/consumer_fraud_probability.csv')
fraud_consumer = spark.createDataFrame(fraud_consumer) 

#Loading merchant fraud data: 
fraud_merchant = pd.read_csv('../data/tables/merchant_fraud_probability.csv')
fraud_merchant = spark.createDataFrame(fraud_merchant) 


In [12]:
full_data = spark.read.parquet('../data/tables/full_join.parquet')

In [5]:
full_data.limit(5)

merchant_name,merchant_abn,categories,take_rate,revenue_levels,name,address,state,gender,trans_merchant_abn,dollar_value,order_id,order_datetime,user_id,consumer_id,postcodes,int_sa2,SA2_code,SA2_name,income_2018-2019,total_males,total_females,total_persons,state_code,state_name,population_2020,population_2021
Dui Nec Corporation,19933438190,tent and awning s...,2.22,c,Brian Wright,761 Bennett Mountain,TAS,Male,19933438190,3.434666831614868,28ccb7e6-ebe7-496...,2021-06-17,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563
Vel Est Tempor LLP,62694031334,"computers, comput...",5.76,a,Brian Wright,761 Bennett Mountain,TAS,Male,62694031334,30.583567099986173,d5902566-aa14-4e4...,2021-05-07,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563
Aliquam Nisl Corp...,55325186693,"gift, card, novel...",4.44,b,Brian Wright,761 Bennett Mountain,TAS,Male,55325186693,68.91163123686927,234aedaa-1657-466...,2021-07-20,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563
Urna Incorporated,78905182348,"watch, clock, and...",4.65,b,Brian Wright,761 Bennett Mountain,TAS,Male,78905182348,15.129648652621713,51d13e41-34a5-496...,2021-06-23,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563
Purus Gravida Sag...,21772962346,florists supplies...,6.63,a,Brian Wright,761 Bennett Mountain,TAS,Male,21772962346,0.7692714085929745,af89a162-38c7-48a...,2021-04-21,2582,743,7016,601021010,601021010,Risdon Vale,58036866,1963,1540,3501,6,Tasmania,3571,3563


The consumer fraud data shows the list of transactions with a high probability of being fraudulent, and therefore should be considered invalid.

In [12]:
print(fraud_consumer.count())
fraud_consumer = fraud_consumer.withColumnRenamed("fraud_probability","cons_fraud_prob").withColumnRenamed("order_datetime","order_datetime_1").withColumnRenamed("user_id","user_id_1")
fraud_consumer.limit(5)

34864


user_id_1,order_datetime_1,cons_fraud_prob
6228,2021-12-19,97.6298077657765
21419,2021-12-10,99.24738020302328
5606,2021-10-17,84.05825045251777
3101,2021-04-17,91.42192091901347
22239,2021-10-19,94.70342477508036


Probability of fraud for high revenue merchants...?

In [13]:
print(fraud_merchant.count())
fraud_merchant = fraud_merchant.withColumnRenamed("merchant_abn","merchant_abn_1").withColumnRenamed("fraud_probability","merch_fraud_prob")
fraud_merchant.limit(5)

114


merchant_abn_1,order_datetime,merch_fraud_prob
19492220327,2021-11-28,44.40365864749536
31334588839,2021-10-02,42.75530083865367
19492220327,2021-12-22,38.867790051131095
82999039227,2021-12-19,94.1347004808891
90918180829,2021-09-02,43.32551731714902


In [14]:

merchant_data = full_data.select("merchant_abn", "categories","revenue_levels","dollar_value","user_id","consumer_id")

In [10]:
merchant_data.limit(5)

merchant_abn,categories,revenue_levels,dollar_value,user_id,consumer_id
19933438190,tent and awning s...,c,3.434666831614868,2582,743
62694031334,"computers, comput...",a,30.583567099986173,2582,743
55325186693,"gift, card, novel...",b,68.91163123686927,2582,743
78905182348,"watch, clock, and...",b,15.129648652621713,2582,743
21772962346,florists supplies...,a,0.7692714085929745,2582,743


- join consumer fraud to final join
- create a version of final join with removed transactions from fraud
- aggregate final join from transaction to merchant level
- add fraud merchant as a feature to aggregated final join
- treat fraud as response and try to create model to give other merchants a fraud score

In [15]:
#Joining consumer fraud, merchant data, and merchant fraud data to one datatset: 

#Joining consumer fraud and merchant data together

merchant_data.createOrReplaceTempView("temp")

fraud_consumer.createOrReplaceTempView("temp2")

join_1 = spark.sql(""" 

SELECT *
FROM temp

LEFT JOIN temp2

ON temp.user_id = temp2.user_id_1 
""")


In [17]:
#Joining join_1 to merchant fraud 

#Joining all test + predicting Sets Together 
join_1.createOrReplaceTempView("temp")

fraud_merchant.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2.merchant_abn_1
""")

In [ ]:
# #Changing all null probability values to 0.01: 

# MIN_P = 0.01
# final_join = final_join.fillna(MIN_P, subset=["cons_fraud_prob", "merch_fraud_prob"])

In [ ]:
#join segment data 
#clustering 
#decide threshold based on clusters 
#use threshold to classify if transaction is fradulent or not 

In [20]:
# Reading in segments 
segments_data = spark.read.csv('../data/curated/tagged_merchants.csv')

In [21]:
segments_data.limit(5)

_c0,_c1,_c2,_c3,_c4,_c5,_c6
null,name,tags,merchant_abn,cleaned_tags,store_type,category
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY


In [22]:
#Removing false header line 
segments_data.na.drop(subset=["_c0"])

_c0,_c1,_c2,_c3,_c4,_c5,_c6
0,Felis Limited,"((furniture, home...",10023283211,furniture home fu...,0,Furniture
1,Arcu Ac Orci Corp...,"([cable, satellit...",10142254217,cable satellite a...,4,Electronics
2,Nunc Sed Company,"([jewelry, watch,...",10165489824,jewelry watch clo...,1,Toys and DIY
3,Ultricies Digniss...,"([wAtch, clock, a...",10187291046,watch clock and j...,1,Toys and DIY
4,Enim Condimentum PC,([music shops - m...,10192359162,music shops musi...,3,"Books, Stationary..."
5,Fusce Company,"[(gift, card, nov...",10206519221,gift card novelty...,4,Electronics
6,Aliquam Enim Inco...,"[(computers, comP...",10255988167,computers compute...,4,Electronics
7,Ipsum Primis Ltd,"[[watch, clock, a...",10264435225,watch clock and j...,1,Toys and DIY
8,Pede Ultrices Ind...,([computer progra...,10279061213,computer programm...,2,"Beauty, Health, P..."
9,Nunc Inc.,"[(furniture, home...",10323485998,furniture home fu...,0,Furniture


In [23]:
final_join.createOrReplaceTempView("temp")

segments_data.createOrReplaceTempView("temp2")

final_join = spark.sql(""" 

SELECT *
FROM temp


LEFT JOIN temp2

ON temp.merchant_abn = temp2._c3
""")

In [49]:
# final_join.limit(5)
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.functions import when
from pyspark.sql import functions as countDistinct

In [1]:
#Creating a table of average fraud probabilities for each category of purchase: 

aggregated_prob = final_join.groupBy("temp2._c6").agg(F.avg("cons_fraud_prob").\
alias("avg_consumer_fraud"),F.avg("merch_fraud_prob").alias("avg_merchant_fraud"))

NameError: name 'final_join' is not defined

In [ ]:
aggregated_prob.show()

In [39]:
#adding a column for is_fraud for each transaction: 

is_fraud = when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085)&(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Toys and DIY") & (final_join["cons_fraud_prob"] > 15.843) &(final_join["merch_fraud_prob"]>32.404),1)\
.when((final_join["_c6"] == "Furniture") & (final_join["cons_fraud_prob"] > 15.077) & (final_join["merch_fraud_prob"]>30.941),1)\
.when((final_join["_c6"] == "Beauty, Health, Personal and Household") & (final_join["cons_fraud_prob"] > 15.392) & (final_join["merch_fraud_prob"]>29.76),1)\
.when((final_join["_c6"] == "Electronics") & (final_join["cons_fraud_prob"] > 15.085) &(final_join["merch_fraud_prob"]>29.635),1)\
.when((final_join["_c6"] == "Books, Stationary and Music") & (final_join["cons_fraud_prob"] > 15.140) &(final_join["merch_fraud_prob"]>29.016),1).otherwise(0)

In [ ]:
aggregated_prob.limit(6)

In [ ]:
final_join.limit(5)

In [42]:
model_with_fraud = final_join.withColumn("is_fraud",is_fraud)

In [52]:
# # model_with_fraud.count()
# merchant_abn_col = model_with_fraud.select("merchant_abn")
# print(merchant_abn_col.count())

17391257


In [ ]:
# model_with_fraud.write.parquet("../data/curated/model_data_with_fraud_col.parquet") 

In [53]:
#Creating a table of average fraud probabilities for each category of purchase: 
final_model_with_fraud = model_with_fraud.groupBy("merchant_abn").agg(F.avg("is_fraud").alias("average fraud rate per merchant"))

In [54]:
final_model_with_fraud.count()

4023

In [55]:
final_model_with_fraud.limit(20)

merchant_abn,average fraud rate per merchant
83412691377,0.0
48214071373,0.0
38700038932,0.0
41956465747,0.0
73256306726,0.0
35344855546,0.0
96946925998,0.0
73841664453,0.0
15613631617,0.0
19839532017,0.0


In [ ]:
final_model_with_fraud.write.parquet("../data/tables/transactions_with_fraud_rates.parquet")

In [ ]:
# data = spark.read.parquet('../data/tables/full_join.parquet')

In [ ]:
# data.limit(5)